<a href="https://colab.research.google.com/github/IvRodco/EstadisticaAplicasa_1899843/blob/main/sesion13y14_G41_14_18sept.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Estadistica aplicada

##Sesion 13, 14 de septiembre

###Ivanna LIzeth Rodriguez Ambriz
###1899843
Grupo 41

In [87]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [12]:
pip install lifelines

In [13]:
from lifelines import CoxPHFitter
import seaborn as sns

In [125]:
df_cancer= pd.read_csv('https://raw.githubusercontent.com/jimmyzac/Estadistica-Aplicada-FCFM-UANL/main/bases_datos/cancer.csv')

In [126]:
#limpiamos o depuramos la base y calculamos estadistica descriptiva
df_cancer.head()

,inst,time,status,age,sex,ph.ecog,ph.karno,pat.karno,meal.cal,wt.loss
0,3.0,306,2,74,1,1.0,90.0,100.0,1175.0,NaN
1,3.0,455,2,68,1,0.0,90.0,90.0,1225.0,15.0
2,3.0,1010,1,56,1,0.0,90.0,90.0,NaN,15.0
3,5.0,210,2,57,1,1.0,90.0,60.0,1150.0,11.0
4,1.0,883,2,60,1,0.0,100.0,90.0,NaN,0.0


In [127]:
#vamos a borrar la variable inst
df_cancer=df_cancer.drop('inst',axis=1)
df_cancer

,time,status,age,sex,ph.ecog,ph.karno,pat.karno,meal.cal,wt.loss
0,306,2,74,1,1.0,90.0,100.0,1175.0,NaN
1,455,2,68,1,0.0,90.0,90.0,1225.0,15.0
2,1010,1,56,1,0.0,90.0,90.0,NaN,15.0
3,210,2,57,1,1.0,90.0,60.0,1150.0,11.0
4,883,2,60,1,0.0,100.0,90.0,NaN,0.0
...,...,...,...,...,...,...,...,...,...
223,188,1,77,1,1.0,80.0,60.0,NaN,3.0
224,191,1,39,1,0.0,90.0,90.0,2350.0,-5.0
225,105,1,75,2,2.0,60.0,70.0,1025.0,5.0
226,174,1,66,1,1.0,90.0,100.0,1075.0,1.0


Los datos constan de 228 observaciones y 10 variables/columnas. La descripción de las variables es la siguiente:
inst: código de institución\
**time (d1)**: tiempo de supervivencia en días\
**status (d2)**: estado de censura 1 = censurado, 2 = muerto\
**age (i1)**: Edad en años\
**sex (i2)**: Masculino = 1 Femenino = 2\
**ph.ecog (i3)**: puntuación de rendimiento ECOG según la calificación del médico. 0 = asintomático, 1 = sintomático pero completamente ambulatorio, 2 = en cama <50% del día, 3 = en cama > 50% del día pero no encamado, 4 = encamado\
**ph.karno (i4)**: puntuación de desempeño de Karnofsky (mala = 0; buena = 100) calificada por el médico\
**pat.karno (i4)**: puntuación de rendimiento de Karnofsky según la calificación del paciente\
**meal.cal (i5)**: Calorías consumidas en las comidas\
**wt.loss (i6)**: Pérdida de peso en los últimos seis meses

In [128]:
#hacer dummies

df_cancer['status']=df_cancer['status']-1
#status=1 son los muertos
#status=0 son los censurados

df_cancer['sex']=df_cancer['sex']-1
#status=1 son femenino
#status=0 son masculino



In [129]:
#vamos a verificar que no existan missing values

#revisamos cuantos missing values tiene cada columna
df_cancer.isnull().sum()


time          0
status        0
age           0
sex           0
ph.ecog       1
ph.karno      1
pat.karno     3
meal.cal     47
wt.loss      14
dtype: int64

In [130]:
#imputamos valores/los eliminamos de la base

#si los eliminamos revisamos cuantos datos perderemos
df_cancer.count()

time         228
status       228
age          228
sex          228
ph.ecog      227
ph.karno     227
pat.karno    225
meal.cal     181
wt.loss      214
dtype: int64

In [131]:

#si barramos los datos eliminariamos una cuarta parte de los datos

#entonces imputaremos valores a los missing values con el promedio
media_ecog= df_cancer['ph.ecog'].mean()
df_cancer['ph.ecog']=df_cancer['ph.ecog'].fillna(media_ecog)

media_phk= df_cancer['ph.karno'].mean()
df_cancer['ph.karno']=df_cancer['ph.karno'].fillna(media_phk)

media_patk= df_cancer['pat.karno'].mean()
df_cancer['pat.karno']=df_cancer['pat.karno'].fillna(media_patk)

media_cal= df_cancer['meal.cal'].mean()
df_cancer['meal.cal']=df_cancer['meal.cal'].fillna(media_cal)

media_loss= df_cancer['wt.loss'].mean()
df_cancer['wt.loss']=df_cancer['wt.loss'].fillna(media_loss)

In [132]:
df_cancer.isnull().sum()

time         0
status       0
age          0
sex          0
ph.ecog      0
ph.karno     0
pat.karno    0
meal.cal     0
wt.loss      0
dtype: int64

In [133]:
df_cancer.dtypes

time           int64
status         int64
age            int64
sex            int64
ph.ecog      float64
ph.karno     float64
pat.karno    float64
meal.cal     float64
wt.loss      float64
dtype: object

#AQUI EMPIEZA LA SESION 14 DEL 18 DE SEPTIEMBRE

In [134]:
##recodificamos ph.ecog como entero
df_cancer['ph.ecog']=df_cancer['ph.ecog'].astype('int64')

In [135]:
#Modelo de Cox (en libreria lifelines) solo admite variables continuas y variables dummies(0,1)

#En la libreria lifelines las variables categoricas se tienen que convertir en dummies (con pandas)

dummies=pd.get_dummies(df_cancer['ph.ecog'], prefix='ecog').astype(int)



In [136]:
dummies.describe()

,ecog_0,ecog_1,ecog_2,ecog_3
count,228.000000,228.000000,228.000000,228.000000
mean,0.280702,0.495614,0.219298,0.004386
std,0.450331,0.501081,0.414681,0.066227
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,1.000000,1.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [141]:
#solo nos quedamos con 3 dummies
dummies=dummies[['ecog_1','ecog_2']]
#la que dejamos fuera es la base(ecog_0)

In [142]:
#unimos dummies con df_cancer

df_cancer=pd.concat([df_cancer,dummies],axis=1)
df_cancer=df_cancer.drop('ph.ecog',axis=1)


In [143]:
#corroborar que sean numericas que que no haya valores perdidos

df_cancer.dtypes

time           int64
status         int64
age            int64
sex            int64
ph.karno     float64
pat.karno    float64
meal.cal     float64
wt.loss      float64
ecog_1         int64
ecog_2         int64
dtype: object

In [144]:
df_cancer.isnull()

,time,status,age,sex,ph.karno,pat.karno,meal.cal,wt.loss,ecog_1,ecog_2
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
223,False,False,False,False,False,False,False,False,False,False
224,False,False,False,False,False,False,False,False,False,False
225,False,False,False,False,False,False,False,False,False,False
226,False,False,False,False,False,False,False,False,False,False


In [145]:
#estadisticas descriptivas

df_cancer.describe()

,time,status,age,sex,ph.karno,pat.karno,meal.cal,wt.loss,ecog_1,ecog_2
count,228.000000,228.000000,228.000000,228.000000,228.000000,228.000000,228.000000,228.000000,228.000000,228.000000
mean,305.232456,0.723684,62.447368,0.394737,81.938326,79.955556,928.779006,9.831776,0.495614,0.219298
std,210.645543,0.448159,9.073457,0.489870,12.300771,14.526227,358.127836,12.728258,0.501081,0.414681
min,5.000000,0.000000,39.000000,0.000000,50.000000,30.000000,96.000000,-24.000000,0.000000,0.000000
25%,166.750000,0.000000,56.000000,0.000000,77.500000,70.000000,768.000000,0.000000,0.000000,0.000000
50%,255.500000,1.000000,63.000000,0.000000,80.000000,80.000000,928.779006,8.000000,0.000000,0.000000
75%,396.500000,1.000000,69.000000,1.000000,90.000000,90.000000,1075.000000,15.000000,1.000000,0.000000
max,1022.000000,1.000000,82.000000,1.000000,100.000000,100.000000,2600.000000,68.000000,1.000000,1.000000


In [146]:
df_cancer['status'].value_counts()

1    165
0     63
Name: status, dtype: int64

###El 72% de los datos han sufrido el evento(morir)

###El 39.6% son mujeres

###El promedio de edad es de 62.4, la edad mas grande es de 82 y la mas joven es de 39

###dado que la media de pat.karno es 79.99 y la de ph.karno es de 82.03 significa que en promedio los pacientes evaluan su estado de salud peor de lo que estan

###La perdida de peso promedio de wt.loss es de 9.73 libras




In [154]:
cph1=CoxPHFitter().fit(df_cancer,'time','status')
cph1.print_summary()

<lifelines.CoxPHFitter: fitted with 228 total observations, 63 right-censored observations>
             duration col = 'time'
                event col = 'status'
      baseline estimation = breslow
   number of observations = 228
number of events observed = 165
   partial log-likelihood = -733.03
         time fit was run = 2023-09-18 21:20:32 UTC

---
            coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                         
age         0.01       1.01       0.01            -0.01             0.03                 0.99                 1.03
sex        -0.58       0.56       0.17            -0.91            -0.25                 0.40                 0.78
ph.karno    0.01       1.01       0.01            -0.01             0.03                 0.99                 1.03
pat.karno  -0.01       0.99       0.01            -0.03             0.00                 0.97                 1.00
meal.cal    0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
wt.loss    -0.01       0.99       0.01            -0.02             0.00                 0.98                 1.00
ecog_1      0.48       1.62       0.23             0.03             0.94                 1.03                 2.57
ecog_2      0.96       2.60       0.37             0.24             1.68                 1.27                 5.34

            cmp to     z      p   -log2(p)
covariate                                 
age           0.00  1.32   0.19       2.42
sex           0.00 -3.40 <0.005      10.54
ph.karno      0.00  1.08   0.28       1.84
pat.karno     0.00 -1.86   0.06       4.00
meal.cal      0.00  0.16   0.87       0.20
wt.loss       0.00 -1.51   0.13       2.92
ecog_1        0.00  2.07   0.04       4.70
ecog_2        0.00  2.61   0.01       6.79
---
Concordance = 0.64
Partial AIC = 1482.06
log-likelihood ratio test = 33.76 on 8 df
-log2(p) of ll-ratio test = 14.44

##Conclusiones

**age**: no se rechaza H0, age no afecta sobre el riesgo de morir de cancer.

**sex**: se rechaza H0, ser mujer disminuye el riesgo de morir de cancer un 40% respecto a los hombres.

**ph.karno**: No se rechaza H0, no tiene efecto sobre el riesgo de morir de cancer

**pat.karno**: No se rechaza H0, no tiene efecto sobre el riesgo de morir de cancer

**meal.cal**: Se rechaza H0, el consumir menos calorias de las necesarias aumenta el riesgo de morir de cancer

**wt.loss**: Se rechaza H0, la perdida de peso en los ultimos 6 meses es una caracteristica principal que marca que ya morira de cancer

**ph.ecog_1**: No se rechaza H0, que ph.ecog = 1, es decir sea sintomático pero completamente ambulatorio no tiene efecto sobre el riesgo de morir de cancer

**ph.ecog_1**: No se rechaza H0, que ph.ecog = 2, es decir 2, es decir que pase en cama < 50% del día no tiene efecto sobre el riesgo de morir de cancer

In [ ]:
#  H0:La propocion de alumnos que pretenden tomar cursos avanzados antes de terminar el curso es la misma al finalizar
#  H1:

In [153]:
(1-0.56)*100

43.99999999999999

El modelo de Cox esta construido sobre el supuesto de riesgo proporcional constante, al estimar el modelo tenemos que comprobar si ese supuesto se cumple o no

In [155]:
#vamos a comprobar el supuesto
#H0:se viola el supuesto
#H1:se cumple el supuesto
cph1.check_assumptions(df_cancer,p_value_threshold=0.05)


The ``p_value_threshold`` is set at 0.05. Even under the null hypothesis of no violations, some
covariates will be below the threshold by chance. This is compounded when there are many covariates.
Similarly, when there are lots of observations, even minor deviances from the proportional hazard
assumption will be flagged.

With that in mind, it's best to use a combination of statistical tests and visual tests to determine
the most serious violations. Produce visual plots using ``check_assumptions(..., show_plots=True)``
and looking for non-constant lines. See link [A] below for a full example.



<lifelines.StatisticalResult: proportional_hazard_test>
 null_distribution = chi squared
degrees_of_freedom = 1
             model = <lifelines.CoxPHFitter: fitted with 228 total observations, 63 right-censored observations>
         test_name = proportional_hazard_test

---
                test_statistic    p  -log2(p)
age       km              0.39 0.53      0.92
          rank            0.17 0.68      0.55
ecog_1    km              3.50 0.06      4.03
          rank            3.40 0.07      3.94
ecog_2    km              3.02 0.08      3.60
          rank            2.58 0.11      3.21
meal.cal  km              4.82 0.03      5.15
          rank            4.21 0.04      4.64
pat.karno km              0.31 0.57      0.80
          rank            0.26 0.61      0.71
ph.karno  km              6.44 0.01      6.48
          rank            5.81 0.02      5.97
sex       km              2.74 0.10      3.36
          rank            2.60 0.11      3.23
wt.loss   km              0.01 0.94      0.09
          rank            0.00 1.00      0.01



1. Variable 'ph.karno' failed the non-proportional test: p-value is 0.0112.

   Advice 1: the functional form of the variable 'ph.karno' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional form.

   Advice 2: try binning the variable 'ph.karno' using pd.cut, and then specify it in
`strata=['ph.karno', ...]` in the call in `.fit`. See documentation in link [B] below.

   Advice 3: try adding an interaction term with your time variable. See documentation in link [C]
below.


2. Variable 'meal.cal' failed the non-proportional test: p-value is 0.0282.

   Advice 1: the functional form of the variable 'meal.cal' might be incorrect. That is, there may
be non-linear terms missing. The proportional hazard test used is very sensitive to incorrect
functional forms. See documentation in link [D] below on how to specify a functional

[]

###Con esto concuilmos que en ph.karno y en meal.cal no se cumple de riesgo constante, entonces como no se cumple el supuesto debemos utilizar un modelo alternativo, el cual es el Modelo de riesgo acelerado (AFT).


###Para esto primero debemos identificar como se distribuye el riesgo (exponencial, weibull o log-Normal)

In [165]:
#revisamos la distribucion del riesgo

#importamos la libreria necesaria para esto
from lifelines import ExponentialFitter, WeibullFitter, LogNormalFitter

mexpo= ExponentialFitter().fit(df_cancer['time'],df_cancer['status'])
mweibul=WeibullFitter().fit(df_cancer['time'],df_cancer['status'])
mlogn=LogNormalFitter().fit(df_cancer['time'],df_cancer['status'])

In [166]:
mexpo.AIC_ , mweibul.AIC_ , mlogn.AIC_

(2326.6763515749335, 2311.7023761788164, 2342.538110633835)

In [167]:
#Dado que con el AIC se elige el de menor valor, eligiremos el Modelo Weibull

#importamos la libreria para el AFT
from lifelines import WeibullAFTFitter

weibulAFT=WeibullAFTFitter().fit(df_cancer,'time','status')
weibulAFT.print_summary()

<lifelines.WeibullAFTFitter: fitted with 228 total observations, 63 right-censored observations>
             duration col = 'time'
                event col = 'status'
   number of observations = 228
number of events observed = 165
           log-likelihood = -1136.96
         time fit was run = 2023-09-18 21:47:39 UTC

---
                    coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
param   covariate                                                                                                         
lambda_ age        -0.01       0.99       0.01            -0.02             0.00                 0.98                 1.00
        ecog_1     -0.36       0.70       0.17            -0.68            -0.03                 0.51                 0.97
        ecog_2     -0.72       0.49       0.26            -1.23            -0.21                 0.29                 0.81
        meal.cal   -0.00       1.00       0.00            -0.00             0.00                 1.00                 1.00
        pat.karno   0.01       1.01       0.01            -0.00             0.02                 1.00                 1.02
        ph.karno   -0.01       0.99       0.01            -0.02             0.00                 0.98                 1.00
        sex         0.42       1.52       0.12             0.18             0.66                 1.19                 1.94
        wt.loss     0.01       1.01       0.00            -0.00             0.02                 1.00                 1.02
        Intercept   6.59     729.24       0.89             4.85             8.33               128.29              4145.29
rho_    Intercept   0.32       1.38       0.06             0.20             0.45                 1.23                 1.56

                    cmp to     z      p   -log2(p)
param   covariate                                 
lambda_ age           0.00 -1.25   0.21       2.25
        ecog_1        0.00 -2.14   0.03       4.94
        ecog_2        0.00 -2.78   0.01       7.51
        meal.cal      0.00 -0.09   0.93       0.11
        pat.karno     0.00  1.89   0.06       4.09
        ph.karno      0.00 -1.29   0.20       2.33
        sex           0.00  3.38 <0.005      10.45
        wt.loss       0.00  1.50   0.13       2.89
        Intercept     0.00  7.44 <0.005      43.12
rho_    Intercept     0.00  5.26 <0.005      22.71
---
Concordance = 0.64
AIC = 2293.92
log-likelihood ratio test = 33.78 on 8 df
-log2(p) of ll-ratio test = 14.45

In [174]:
(1-1.01)*100

-1.0000000000000009